In [42]:
import re
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [43]:
def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
    return df


In [44]:
def class_id_to_label(id):
    label_map = {1: 'glass', 2: 'paper', 3: 'cardboard', 4: 'plastic', 5: 'metal', 6: 'trash'}
    return label_map[id]

In [45]:
IMAGES_DIR = '/kaggle/input/garbage-classification/Garbage classification/Garbage classification/'
train_file = '/kaggle/input/garbage-classification/one-indexed-files-notrash_train.txt'
val_file   = '/kaggle/input/garbage-classification/one-indexed-files-notrash_val.txt'
test_file  = '/kaggle/input/garbage-classification/one-indexed-files-notrash_test.txt'

In [46]:
df_train = pd.read_csv(train_file, sep=' ', header=None, names=['rel_path', 'label'])
df_valid = pd.read_csv(val_file,   sep=' ', header=None, names=['rel_path', 'label'])
df_test  = pd.read_csv(val_file,   sep=' ', header=None, names=['rel_path', 'label'])

In [47]:
df_train = add_class_name_prefix(df_train, 'rel_path')
df_valid = add_class_name_prefix(df_valid, 'rel_path')
df_test  = add_class_name_prefix(df_test,  'rel_path')

In [48]:
df_train['label'] = df_train['label'].apply(class_id_to_label)
df_valid['label'] = df_valid['label'].apply(class_id_to_label)
df_test['label']  = df_test['label'].apply(class_id_to_label)

In [49]:
df_train

,rel_path,label
0,cardboard/cardboard202.jpg,cardboard
1,paper/paper472.jpg,paper
2,paper/paper522.jpg,paper
3,glass/glass189.jpg,glass
4,glass/glass325.jpg,glass
...,...,...
1763,cardboard/cardboard6.jpg,cardboard
1764,glass/glass283.jpg,glass
1765,metal/metal335.jpg,metal
1766,plastic/plastic133.jpg,plastic


In [50]:
df_valid

,rel_path,label
0,cardboard/cardboard114.jpg,cardboard
1,plastic/plastic204.jpg,plastic
2,glass/glass123.jpg,glass
3,glass/glass152.jpg,glass
4,glass/glass398.jpg,glass
...,...,...
323,metal/metal58.jpg,metal
324,cardboard/cardboard149.jpg,cardboard
325,plastic/plastic5.jpg,plastic
326,glass/glass23.jpg,glass


In [51]:
df_test

,rel_path,label
0,cardboard/cardboard114.jpg,cardboard
1,plastic/plastic204.jpg,plastic
2,glass/glass123.jpg,glass
3,glass/glass152.jpg,glass
4,glass/glass398.jpg,glass
...,...,...
323,metal/metal58.jpg,metal
324,cardboard/cardboard149.jpg,cardboard
325,plastic/plastic5.jpg,plastic
326,glass/glass23.jpg,glass


In [52]:
datagen = ImageDataGenerator()

In [53]:
datagen_train = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=IMAGES_DIR,
    x_col='rel_path',
    y_col='label',
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=7,
)

datagen_valid = datagen.flow_from_dataframe(
    dataframe=df_valid,
    directory=IMAGES_DIR,
    x_col='rel_path',
    y_col='label',
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=7,
)

Found 1768 validated image filenames belonging to 6 classes.
Found 328 validated image filenames belonging to 6 classes.


In [54]:
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False
        
    return model

In [55]:
net = build_model(num_classes=6)

In [56]:
net.compile(optimizer='Adam',
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.categorical_accuracy])

### Training and saving done before refresh

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

history = net.fit_generator(
    generator=datagen_train,
    validation_data=datagen_valid,
    epochs=30,
    validation_freq=1,
    callbacks=[early_stop]
)


In [ ]:
net

In [ ]:
net.save('gar_nn_model1.h5')

### Loading saved model

In [57]:
loaded_model = tf.keras.models.load_model('./gar_nn_model1.h5')


In [58]:
loaded_model

In [59]:
test_generator = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=IMAGES_DIR,
    x_col='rel_path',
    y_col='label',
    color_mode="rgb",
    class_mode="categorical",
    batch_size=1,
    shuffle=False,
    seed=7
)

Found 328 validated image filenames belonging to 6 classes.


In [60]:
filenames = test_generator.filenames

In [62]:
nb_samples = len(filenames)
nb_samples

328

In [65]:
loaded_model.evaluate_generator(test_generator, nb_samples)

[0.27755361795425415, 0.8963414430618286]